In [39]:
from bs4 import BeautifulSoup as bs
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import steam_review_scraper 
import requests
from datetime import datetime as dt
import pandas as pd
from steam_review_scraper import get_game_review 
import pickle
import steamreviews

In [ ]:
#https://steamcommunity.com/app/1517290/reviews/?browsefilter=mostrecent&snr=1_5_100010_&p=1 steam website
driver = webdriver.Chrome("chromedriver.exe") 
driver.get("https://steamcommunity.com/app/1517290/reviews/?browsefilter=mostrecent&snr=1_5_100010_&p=1")#navgiate to website
time.sleep(1)
element=driver.find_element(By.CLASS_NAME,"btn_grey_white_innerfade") # find element
element.click() #click on element

In [ ]:
#open drop down list for review ratings
element= driver.find_element(By.CLASS_NAME, "user_reviews_rating_filter")
element.click()

In [ ]:
#click on positive reviews only
driver.find_element(By.ID,"filterrating_option_1").click()

In [ ]:
pre_scroll_height = driver.execute_script('return document.body.scrollHeight;')
run_time, max_run_time = 0, 2 

while True:
    iteration_start = time.time() #start time
    driver.execute_script('window.scrollTo(0, 100*document.body.scrollHeight);') # scroll to the end of the page

    post_scroll_height = driver.execute_script('return document.body.scrollHeight;') #retrun the height of the page after scrolling

    scrolled = post_scroll_height != pre_scroll_height #to check if post scroll and pre scroll height are the same
    timed_out = run_time >= max_run_time #time does not go over 2 seconds when loading page

    if scrolled:
        run_time = 0 
        pre_scroll_height = post_scroll_height
    elif not scrolled and not timed_out:
         run_time += time.time() - iteration_start
    elif not scrolled and timed_out:
         break
    try:
        element = driver.find_element(By.ID,"GetMoreContentBtn")
        element.click()
    except:
        continue

In [ ]:
#scrape date from loaded website
soup = bs(driver.page_source, "lxml")

In [ ]:
#write the website content to a local file
with open("output1.html", "w", encoding='utf-8') as file:
    file.write(str(soup))

In [ ]:
#open drop down list for review ratings
element= driver.find_element(By.CLASS_NAME, "user_reviews_rating_filter")
element.click()

In [ ]:
#click on negative reviews only
driver.find_element(By.ID,"filterrating_option_2").click()

In [ ]:
pre_scroll_height = driver.execute_script('return document.body.scrollHeight;')
run_time, max_run_time = 0, 2 

while True:
    iteration_start = time.time() #start time
    driver.execute_script('window.scrollTo(0, 100*document.body.scrollHeight);') # scroll to the end of the page

    post_scroll_height = driver.execute_script('return document.body.scrollHeight;') #retrun the height of the page after scrolling

    scrolled = post_scroll_height != pre_scroll_height #to check if post scroll and pre scroll height are the same
    timed_out = run_time >= max_run_time #time does not go over 2 seconds when loading page

    if scrolled:
        run_time = 0 
        pre_scroll_height = post_scroll_height
    elif not scrolled and not timed_out:
         run_time += time.time() - iteration_start
    elif not scrolled and timed_out:
         break
    try:
        element = driver.find_element(By.ID,"GetMoreContentBtn")
        element.click()
    except:
        continue

In [ ]:
#scrape date from loaded website
soup = bs(driver.page_source, "lxml")

In [ ]:
#write the website content to a local file
with open("output1.html", "w", encoding='utf-8') as file:
    file.write(str(soup))

In [33]:
#open local hmtl file which contains all postive reviews
HTMLFileToBeOpened = open("output1.html", "r",encoding="utf8")
contents = HTMLFileToBeOpened.read()
beautifulSoupText = bs(contents, 'lxml')

In [37]:
beautifulSoupText.find_all("div", {"class": "apphub_UserReviewCardContent"})[1]

<div class="apphub_UserReviewCardContent">
<div class="found_helpful">
				No one has rated this review as helpful yet				<div class="review_award_aggregated tooltip" data-tooltip-class="review_reward_tooltip" data-tooltip-html='&lt;div class="review_award_ctn_hover"&gt;This item has no awards&lt;/div&gt;'><img class="reward_btn_icon" src="https://community.akamai.steamstatic.com/public/shared/images//award_icon_blue.svg"/>0</div>
</div>
<div class="vote_header">
<div class="reviewInfo">
<div class="thumb">
<img height="44" src="https://community.akamai.steamstatic.com/public/shared/images/userreviews/icon_thumbsUp.png?v=1" width="44"/>
</div>
<div class="title">Recommended</div>
<div class="hours">1.0 hrs on record</div>
</div>
<div style="clear: left"></div>
</div>
<div class="apphub_CardTextContent">
<div class="date_posted">Posted: 27 January</div>
																surprisingly good! Shields are OP :) one hit kill LOL			</div>
</div>

In [ ]:
with open("output2.html", "w", encoding='utf-8') as file:
    file.write(str(soup))

In [17]:
#open local file
HTMLFileToBeOpened = open("output2.html", "r",encoding="utf8")
contents = HTMLFileToBeOpened.read()
beautifulSoupText = bs(contents, 'lxml')

In [32]:
beautifulSoupText.find_all("div", {"class": "apphub_CardTextContent"})[25449]

<div class="apphub_CardTextContent">
<div class="date_posted">Posted: December 18, 2021</div>
																The game is honestly really fun, the guns feel great, and the classes all feel refreshing. The only issue is the stability of the game, and how getting hit with a car can crash your game. I'd hold out until the game is more polished.			</div>

In [41]:
(((50000/150)*310)/60)/60

28.703703703703702